# Node Reference Guide

**Node** is the polymorphic container in lionherd - extends Element with structured content, embeddings, and automatic type registry.

## Key Features

1. **Element Inheritance**: Inherits `id`, `metadata`, `created_at` from Element base class
2. **Structured Content**: `content: dict | Serializable | BaseModel | None` enforces query-able, composable data (breaking change in Alpha3)
3. **Embedding Support**: Optional `embedding: list[float]` with DB compatibility (JSON string coercion)
4. **Auto-Registry**: Subclasses auto-register via `__pydantic_init_subclass__` for polymorphic deserialization
5. **Serialization Modes**: `python`/`json`/`db` modes with automatic lion_class injection
6. **Pydapter Integration**: TOML/YAML adapters with isolated per-subclass registries (Rust-like explicit pattern)

## Design Philosophy

- **Composition layer**: Node is structured composition vs Element (identity) vs Event (execution)
- **JSONB one-stop-shop**: All content types guarantee PostgreSQL query-ability
- **Pit of success**: Structured constraint forces graph-thinking, prevents primitive soup
- **Zero-config subclasses**: Registry eliminates boilerplate - subclasses "just work"
- **Database-first**: `mode="db"` uses `node_metadata` to avoid column conflicts

## ⚠️ Alpha3 Breaking Change

**Content constraint** (PR #48): Primitives (str, int, float, etc.) **no longer accepted**.

**Migration**:
```python
# ❌ Before (primitives accepted)
Node(content="test")
Node(content=42)

# ✅ After (structured types only)
Node(content={"value": "test"})
Node(content={"count": 42})

# ✅ Or use Element.metadata for simple key-value
Element(metadata={"text": "test"})
```

## Setup

In [1]:
from lionherd_core.base.node import NODE_REGISTRY, Node


# Define sample Node subclasses (auto-register)
class PersonNode(Node):
    """Node representing a person."""

    name: str = "Unknown"
    age: int = 0


class DocumentNode(Node):
    """Node representing a document."""

    title: str = "Untitled"
    body: str = ""

## 1. Element Inheritance

Node extends Element, inheriting automatic ID generation, metadata, and timestamps.

In [2]:
# Create base Node - inherits Element features
node = Node(content={"text": "Hello World"})

print(f"ID (UUID): {node.id}")
print(f"Created at: {node.created_at}")
print(f"Metadata: {node.metadata}")
print(f"Content: {node.content}")
print(f"\nClass name: {node.class_name(full=True)}")

ID (UUID): 2046ebe0-4f9e-4d43-bfb5-e7f87a08fe3f
Created at: 2025-11-10 20:38:13.643053+00:00
Metadata: {}
Content: {'text': 'Hello World'}

Class name: lionherd_core.base.node.Node


## 2. Content Constraint (Structured Types Only)

The `content` field accepts **structured types only**: `dict`, `Serializable`, `BaseModel`, or `None`.

**Why**: Enforces query-able, composable data for PostgreSQL JSONB and graph-of-graphs patterns.

**Rejected**: Primitives (str, int, float, bool), collections (list, tuple, set)

In [3]:
# Structured content examples
node_dict = Node(content={"key": "value", "nested": [1, 2, 3]})
node_wrapped = Node(content={"items": ["a", "b", "c"]})

# Content can be nested Node (graph-of-graphs pattern)
inner = PersonNode(name="Alice", age=30)
outer = Node(content=inner)

print(f"Dict content: {node_dict.content}")
print(f"Wrapped list: {node_wrapped.content}")
print(f"Nested Node: {outer.content.name}, age {outer.content.age}")

# ❌ Primitives rejected with helpful error
try:
    Node(content="plain string")
except TypeError as e:
    print(f"\n❌ Primitive rejected: {str(e)[:80]}...")

Dict content: {'key': 'value', 'nested': [1, 2, 3]}
Wrapped list: {'items': ['a', 'b', 'c']}
Nested Node: Alice, age 30

❌ Primitive rejected: content must be Serializable, BaseModel, dict, or None. Got str. Use dict for un...


## 3. Nested Element Serialization

When content contains Elements, they auto-serialize via `_serialize_content` field serializer.

In [4]:
# Create Node with nested PersonNode
person = PersonNode(name="Bob", age=25, content={"role": "engineer"})
wrapper = Node(content=person)

# Serialize - content becomes dict automatically
data = wrapper.to_dict()
print(f"Serialized content type: {type(data['content'])}")
print(f"Content includes lion_class: {'lion_class' in data['content']['metadata']}")

# Deserialize - content becomes PersonNode automatically
restored = Node.from_dict(data)
print(f"\nRestored content type: {type(restored.content)}")
print(f"Restored name: {restored.content.name}")
print(f"Restored role: {restored.content.content['role']}")

Serialized content type: <class 'dict'>
Content includes lion_class: True

Restored content type: <class '__main__.PersonNode'>
Restored name: Bob
Restored role: engineer


## 4. Embedding Field

Optional `embedding: list[float]` with validation and JSON string coercion for DB compatibility.

In [5]:
# Embedding as list (standard)
node1 = Node(content={"text": "sample"}, embedding=[0.1, 0.2, 0.3])
print(f"Embedding: {node1.embedding}")

# Embedding from JSON string (DB compatibility)
import orjson

json_str = orjson.dumps([0.4, 0.5, 0.6]).decode()
node2 = Node(content={"text": "sample"}, embedding=json_str)
print(f"From JSON string: {node2.embedding}")

# Integer coercion to float
node3 = Node(content={"text": "sample"}, embedding=[1, 2, 3])
print(f"Int coerced to float: {node3.embedding}")
print(f"All floats: {all(isinstance(x, float) for x in node3.embedding)}")

Embedding: [0.1, 0.2, 0.3]
From JSON string: [0.4, 0.5, 0.6]
Int coerced to float: [1.0, 2.0, 3.0]
All floats: True


## 5. NODE_REGISTRY Auto-Registration

Subclasses auto-register via `__pydantic_init_subclass__` - enables polymorphic deserialization.

In [6]:
# Check registry
print("Registry keys:", list(NODE_REGISTRY.keys()))
print(f"\nPersonNode registered: {'PersonNode' in NODE_REGISTRY}")
print(f"DocumentNode registered: {'DocumentNode' in NODE_REGISTRY}")


# Dynamic subclass registration
class CustomNode(Node):
    custom_field: str = "test"


print(f"\nCustomNode auto-registered: {'CustomNode' in NODE_REGISTRY}")
print(f"Registry returns correct class: {NODE_REGISTRY['CustomNode'] is CustomNode}")

Registry keys: ['Node', 'lionherd_core.base.node.Node', 'PersonNode', '__main__.PersonNode', 'DocumentNode', '__main__.DocumentNode']

PersonNode registered: True
DocumentNode registered: True

CustomNode auto-registered: True
Registry returns correct class: True


## 6. Polymorphic Deserialization

`from_dict()` uses `lion_class` metadata to route to correct subclass from NODE_REGISTRY.

In [7]:
# Serialize different node types
person = PersonNode(name="Charlie", age=35)
doc = DocumentNode(title="Spec", body="Requirements")

person_data = person.to_dict()
doc_data = doc.to_dict()

print(f"Person lion_class: {person_data['metadata']['lion_class']}")
print(f"Doc lion_class: {doc_data['metadata']['lion_class']}")

# Deserialize via base Node.from_dict() - polymorphic routing
restored_person = Node.from_dict(person_data)
restored_doc = Node.from_dict(doc_data)

print(f"\nRestored person type: {type(restored_person).__name__}")
print(f"Restored doc type: {type(restored_doc).__name__}")
print(f"Person name: {restored_person.name}")
print(f"Doc title: {restored_doc.title}")

Person lion_class: __main__.PersonNode
Doc lion_class: __main__.DocumentNode

Restored person type: PersonNode
Restored doc type: DocumentNode
Person name: Charlie
Doc title: Spec


## 7. Heterogeneous Collections (Real-World DB Scenario)

Single query returns mixed node types - polymorphic deserialization preserves types.

In [8]:
# Simulate DB query returning mixed types with node_metadata
db_records = [
    {"name": "Alice", "age": 30, "node_metadata": {"lion_class": "PersonNode"}},
    {"title": "Report", "body": "Data", "node_metadata": {"lion_class": "DocumentNode"}},
    {"name": "Bob", "age": 25, "node_metadata": {"lion_class": "PersonNode"}},
]

# Deserialize - each gets correct type
nodes = [Node.from_dict(record) for record in db_records]

print("Deserialized types:")
for i, node in enumerate(nodes):
    print(f"  [{i}] {type(node).__name__}", end="")
    if isinstance(node, PersonNode):
        print(f" - {node.name}, age {node.age}")
    elif isinstance(node, DocumentNode):
        print(f" - {node.title}")

Deserialized types:
  [0] PersonNode - Alice, age 30
  [1] DocumentNode - Report
  [2] PersonNode - Bob, age 25


## 8. Serialization Modes

Three modes: `python` (in-memory), `json` (APIs), `db` (database with node_metadata).

In [9]:
node = PersonNode(name="David", age=40)

# Python mode - preserves datetime/UUID objects
python_dict = node.to_dict(mode="python")
print(f"Python mode has 'metadata': {'metadata' in python_dict}")
print(f"Created_at type: {type(python_dict['created_at']).__name__}")

# JSON mode - serializes to strings
json_dict = node.to_dict(mode="json")
print(f"\nJSON mode has 'metadata': {'metadata' in json_dict}")
print(f"Created_at type: {type(json_dict['created_at']).__name__}")

# DB mode - uses node_metadata (avoids column conflicts)
db_dict = node.to_dict(mode="db")
print(f"\nDB mode has 'node_metadata': {'node_metadata' in db_dict}")
print(f"DB mode has 'metadata': {'metadata' in db_dict}")
print(f"lion_class in node_metadata: {'lion_class' in db_dict['node_metadata']}")

Python mode has 'metadata': True
Created_at type: datetime

JSON mode has 'metadata': True
Created_at type: str

DB mode has 'node_metadata': True
DB mode has 'metadata': False
lion_class in node_metadata: True


## 9. Serialization Roundtrip (All Modes)

All modes support lossless roundtrip with polymorphic type preservation.

In [10]:
original = DocumentNode(title="Architecture", body="System design")

for mode in ["python", "json", "db"]:
    # Serialize
    data = original.to_dict(mode=mode)

    # Deserialize
    restored = Node.from_dict(data)

    # Verify
    print(
        f"{mode:8} -> Type: {type(restored).__name__:12} Title: {restored.title:15} ID match: {restored.id == original.id}"
    )

python   -> Type: DocumentNode Title: Architecture    ID match: True
json     -> Type: DocumentNode Title: Architecture    ID match: True
db       -> Type: DocumentNode Title: Architecture    ID match: True


## 10. Pydapter Integration (Isolated Registry Pattern)

Base Node has TOML/YAML built-in. Subclasses get **isolated registries** (Rust-like explicit) - must register adapters explicitly.

In [11]:
from pydapter.adapters import TomlAdapter

# Base Node has toml/yaml built-in
base_node = Node(content={"value": "test"})
toml_str = base_node.adapt_to("toml")
print("Base Node TOML (first 100 chars):")
print(toml_str[:100])

# Subclasses have ISOLATED registries - must register explicitly
PersonNode.register_adapter(TomlAdapter)
person = PersonNode(name="Eve", age=28)
person_toml = person.adapt_to("toml")
print("\nPersonNode TOML (first 100 chars):")
print(person_toml[:100])

# Roundtrip with polymorphism
restored = Node.adapt_from(person_toml, "toml")
print(f"\nRestored type: {type(restored).__name__}")
print(f"Restored name: {restored.name}")

Base Node TOML (first 100 chars):
id = "4518353d-978a-445a-9a43-27658bf6fdd7"
created_at = 2025-11-10T20:38:13.703670Z

[content]
valu

PersonNode TOML (first 100 chars):
id = "a98cc9bb-2345-4a0f-a80b-32d5427f9df3"
created_at = 2025-11-10T20:38:13.703875Z
name = "Eve"
ag

Restored type: PersonNode
Restored name: Eve


## 11. Advanced: created_at_format Options

Control timestamp serialization: `datetime` (object), `isoformat` (string), `timestamp` (float).

In [12]:
node = Node(content={"value": "test"})

# Datetime format (default for python mode)
data1 = node.to_dict(mode="python", created_at_format="datetime")
print(f"datetime format: {type(data1['created_at']).__name__} = {data1['created_at']}")

# ISO format (string)
data2 = node.to_dict(mode="python", created_at_format="isoformat")
print(f"isoformat: {type(data2['created_at']).__name__} = {data2['created_at']}")

# Timestamp format (float)
data3 = node.to_dict(mode="python", created_at_format="timestamp")
print(f"timestamp: {type(data3['created_at']).__name__} = {data3['created_at']}")

datetime format: datetime = 2025-11-10 20:38:13.712678+00:00
isoformat: str = 2025-11-10T20:38:13.712678+00:00
timestamp: float = 1762807093.712678


## 12. Advanced: Custom Content Serialization

The `content_serializer` parameter enables custom transformation of content during serialization without modifying Node behavior.

**Use Cases**: Compression, encryption, external storage references, format conversion.

## Summary Checklist

### Core Patterns Demonstrated

- ✅ **Element Inheritance**: Node inherits `id`, `metadata`, `created_at` from Element
- ✅ **Structured Content**: `content: dict | Serializable | BaseModel | None` enforces query-able data
- ✅ **Nested Serialization**: Elements in content auto-serialize via `_serialize_content`
- ✅ **Embedding Field**: `list[float]` with JSON string coercion + int→float coercion
- ✅ **Auto-Registry**: `__pydantic_init_subclass__` registers subclasses in `NODE_REGISTRY`
- ✅ **Polymorphic Deserialization**: `from_dict()` routes via `lion_class` metadata
- ✅ **Heterogeneous Collections**: Single `Node.from_dict()` call handles mixed types
- ✅ **Serialization Modes**: `python`/`json`/`db` with `node_metadata` for DB
- ✅ **Pydapter Integration**: Isolated adapter registries (Rust-like explicit)
- ✅ **Timestamp Control**: `created_at_format` options (datetime/isoformat/timestamp)
- ✅ **Custom Content Serialization**: `content_serializer` parameter for compression, external refs, format conversion

### Real-World Use Cases

1. **Graph Databases**: Heterogeneous node types in single query → polymorphic deserialization
2. **Nested Composition**: Node contains Graph contains Nodes (graph-of-graphs)
3. **Vector Search**: Embedding field for semantic search with DB JSON compatibility
4. **External Formats**: TOML/YAML serialization with type preservation
5. **DB Storage**: `mode="db"` with `node_metadata` avoids column conflicts
6. **Content Transformation**: Custom serializers for compression, encryption, external storage

### Key Design Decisions

- **Structured content constraint (Alpha3)**: Enforces JSONB query-ability, prevents primitive soup
- **Composition layer identity**: Node (structured) vs Element (identity) vs Event (execution)
- **Zero-config subclasses**: Registry eliminates manual registration boilerplate
- **Isolated adapter registries**: Prevents pollution, explicit > implicit
- **Database-first design**: JSON string coercion, node_metadata field, serialization modes
- **Custom serialization**: One-way content transformation without modifying Node behavior

### Migration from Pre-Alpha3

```python
# ❌ Before: Primitives accepted
Node(content="text")
Node(content=42)
Node(content=["a", "b", "c"])

# ✅ After: Structured types only
Node(content={"text": "value"})
Node(content={"count": 42})
Node(content={"items": ["a", "b", "c"]})

# ✅ Or use Element.metadata for simple key-value
Element(metadata={"text": "value"})
```

## Summary Checklist

### Core Patterns Demonstrated

- ✅ **Element Inheritance**: Node inherits `id`, `metadata`, `created_at` from Element
- ✅ **Structured Content**: `content: dict | Serializable | BaseModel | None` enforces query-able data
- ✅ **Nested Serialization**: Elements in content auto-serialize via `_serialize_content`
- ✅ **Embedding Field**: `list[float]` with JSON string coercion + int→float coercion
- ✅ **Auto-Registry**: `__pydantic_init_subclass__` registers subclasses in `NODE_REGISTRY`
- ✅ **Polymorphic Deserialization**: `from_dict()` routes via `lion_class` metadata
- ✅ **Heterogeneous Collections**: Single `Node.from_dict()` call handles mixed types
- ✅ **Serialization Modes**: `python`/`json`/`db` with `node_metadata` for DB
- ✅ **Pydapter Integration**: Isolated adapter registries (Rust-like explicit)
- ✅ **Timestamp Control**: `created_at_format` options (datetime/isoformat/timestamp)

### Real-World Use Cases

1. **Graph Databases**: Heterogeneous node types in single query → polymorphic deserialization
2. **Nested Composition**: Node contains Graph contains Nodes (graph-of-graphs)
3. **Vector Search**: Embedding field for semantic search with DB JSON compatibility
4. **External Formats**: TOML/YAML serialization with type preservation
5. **DB Storage**: `mode="db"` with `node_metadata` avoids column conflicts

### Key Design Decisions

- **Structured content constraint (Alpha3)**: Enforces JSONB query-ability, prevents primitive soup
- **Composition layer identity**: Node (structured) vs Element (identity) vs Event (execution)
- **Zero-config subclasses**: Registry eliminates manual registration boilerplate
- **Isolated adapter registries**: Prevents pollution, explicit > implicit
- **Database-first design**: JSON string coercion, node_metadata field, serialization modes

### Migration from Pre-Alpha3

```python
# ❌ Before: Primitives accepted
Node(content="text")
Node(content=42)
Node(content=["a", "b", "c"])

# ✅ After: Structured types only
Node(content={"text": "value"})
Node(content={"count": 42})
Node(content={"items": ["a", "b", "c"]})

# ✅ Or use Element.metadata for simple key-value
Element(metadata={"text": "value"})
```